In [1]:
library("cmapR")
library("RCSM")
library("tibble")
library("dplyr")

Warning message:
“package ‘cmapR’ was built under R version 4.4.3”
Warning message:
“package ‘tibble’ was built under R version 4.4.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
trans <- read.table("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/0.Data/LINCS_beta/geneinfo_beta.txt", header=T, sep="\t",quote = "")
colnames(trans)[2] <- "X"

In [3]:

drugs<-read.table("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/0.Data/NemoCad_output/drugs_in_full_signature_PLSR.txt", header=F, sep="\t",quote = "")
drugs<-drugs$V1
#

In [4]:
sig <- read.table("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/0.Data/LINCS_beta/siginfo_beta.txt", header=T, sep="\t",quote = "")


In [5]:
sig<-sig[sig$cmap_name %in% drugs,]
# considering only relevant cell types to reduce the dimesnion of this?
relevant_cell<-unique(sig$cell_mfc_name[grepl("NEU|NPC",sig$cell_mfc_name)])
relevant_cell<-relevant_cell[!relevant_cell=="FIBRNPC"]
sig<-sig[sig$cell_mfc_name %in% relevant_cell,]


In [6]:
signatures_parse<-unique(sig$sig_id)
length(signatures_parse)

[1] 2453

In [8]:
my_ds <- parse_gctx("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/0.Data/LINCS_beta/level5_beta_trt_cp_n720216x12328.gctx",cid=signatures_parse)
m <- mat(my_ds) #the parsing should be on the interesitng drugs

reading /mnt/data/RNAseq_pipeline/ConnectivityMap/LINCS/beta/level5_beta_trt_cp_n720216x12328.gctx

done



In [9]:
final_signatures<-colnames(m)

In [10]:
sig<-sig[sig$sig_id %in% final_signatures,]


In [ ]:
setdiff(drugs,unique(sig$cmap_name))

In [13]:
#signatures<-read.csv("/mnt/data/RNAseq_pipeline/ConnectivityMap/data/all_results_nemocad_MM0Accuracy_top6.csv")
signatures<-read.csv("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Analysis/Drug_repurposing/all_results_nemocad_PLSR.csv")
clusters<-levels(factor(signatures$cluster))
clusters

[1] "clust1" "clust2" "clust3" "clust4"

In [19]:
results<-list()
for (cluster in clusters){


    Up<-signatures[signatures$cluster==cluster & signatures$regulation==1,]#hier_clus
    Up$X=Up$gene
    Up_trans<-merge(trans, Up, by="X")
    Down<-signatures[signatures$cluster==cluster & signatures$regulation==0,]
    Down$X=Down$gene
    Down_trans<-merge(trans, Down, by="X")
    
    cmap1 <- GSEAweight0Score(m, as.character(Up_trans[,2]), as.character(Down_trans[,2]), permuteNum = 1,pAdjMethod = "BH", mcCore = 15)
    #print("ready with cmap1")
    cmap2 <- GSEAweight1Score(m, as.character(Up_trans[,2]), as.character(Down_trans[,2]), permuteNum = 1,pAdjMethod = "BH", mcCore = 15)
    #print("ready with cmap2")

    cmap3 <- GSEAweight2Score(m, as.character(Up_trans[,2]), as.character(Down_trans[,2]), permuteNum = 1,pAdjMethod = "BH", mcCore = 15)
    #print("ready with cmap3")

    cmap4 <- ZhangScore(m, as.character(Up_trans[,2]), as.character(Down_trans[,2]), permuteNum = 1,pAdjMethod = "BH", mcCore = 15)
    #print("ready with cmap4")

    cmap5 <- KSScore(m, as.character(Up_trans[,2]), as.character(Down_trans[,2]), permuteNum = 1,pAdjMethod = "BH", mcCore = 15)
    #print("ready with cmap5")

    cmap6 <- XSumScore(m, as.character(Up_trans[,2]), as.character(Down_trans[,2]), permuteNum = 1,pAdjMethod = "BH", mcCore = 15)
    print("ready with cmap6")

    cmap1 <- rownames_to_column(cmap1)
    cmap2 <- rownames_to_column(cmap2)
    cmap3 <- rownames_to_column(cmap3)
    cmap4 <- rownames_to_column(cmap4)
    cmap5 <- rownames_to_column(cmap5)
    cmap6 <- rownames_to_column(cmap6)

    colnames(cmap1) <- c("sig_id","GSEAweight0Score","GSEAweight0Score_P","GSEAweight0Score_adjP")
    colnames(cmap2) <- c("sig_id","GSEAweight1Score","GSEAweight1Score_P","GSEAweight1Score_adjP")
    colnames(cmap3) <- c("sig_id","GSEAweight2Score","GSEAweight2Score_P","GSEAweight2Score_adjP")
    colnames(cmap4) <- c("sig_id","ZhangScore","ZhangScore_P","ZhangScore_adjP")
    colnames(cmap5) <- c("sig_id","KSScore","KSScore_P","KSScore_adjP")
    colnames(cmap6) <- c("sig_id","XSumScore","XSumScore_P","XSumScore_adjP")

    merge <- merge(cmap1, cmap2, by="sig_id")
    merge <- merge(merge, cmap3, by="sig_id")
    merge <- merge(merge, cmap4, by="sig_id")
    merge <- merge(merge, cmap5, by="sig_id")
    merge <- merge(merge, cmap6, by="sig_id")

    merge <- mutate(merge, mean_col = rowMeans(select(merge,c("GSEAweight0Score","GSEAweight1Score","GSEAweight2Score","ZhangScore"
    ,"KSScore","XSumScore")), na.rm = TRUE))
    
    merge <- merge(sig, merge, by="sig_id")
    #merge <- merge(pert, merge, by="pert_id")
    #dim(merge)
    results[[cluster]]<-merge

    }


[1] "ready with cmap6"
[1] "ready with cmap6"
[1] "ready with cmap6"
[1] "ready with cmap6"


In [20]:
names(results)<-clusters

results_2<-results
for (cl in names(results_2)){
    df<-results_2[[cl]]
    df$cluster<-cl
    results_2[[cl]]<-df
    }

all_results<-do.call(rbind, results_2)
all_results<- all_results %>% mutate(across(c("GSEAweight0Score","GSEAweight1Score","GSEAweight2Score","ZhangScore",
    "KSScore","XSumScore"), scale, .names = "scaled_{.col}")) %>%
    mutate(mean_col_scaled = rowMeans(select(., starts_with("scaled_")), na.rm=TRUE))
write.csv(all_results, "T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Analysis/Drug_repurposing/results/nemocad_RCSM_drugs_predictions.csv")    